In [10]:
import logging
#logging.getLogger("imperative_model").setLevel(logging.DEBUG)
#logging.basicConfig(level=logging.DEBUG)

In [11]:
%run repl_load_model.ipy

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
model.processes

[Process(id='AmmoniaSynthesis', produces=['Ammonia'], consumes=['Hydrogen', 'Nitrogen'], has_stock=False),
 Process(id='ExtractCO2FromAtmosphere', produces=['CO2'], consumes=['AtmosphericCO2'], has_stock=False),
 Process(id='HydrogenSynthesisFromCoal', produces=['CO2', 'Hydrogen'], consumes=['Coal', 'Water'], has_stock=False),
 Process(id='HydrogenSynthesisFromNaturalGas', produces=['CO2', 'Hydrogen'], consumes=['NaturalGas', 'Water'], has_stock=False),
 Process(id='ProducingNitricAcid', produces=['NitricAcid', 'Water'], consumes=['Ammonia', 'PureOxygen', 'Water'], has_stock=False),
 Process(id='ProducingUrea', produces=['OtherIndustrialGases', 'Urea', 'Water'], consumes=['CO2', 'Ammonia'], has_stock=False),
 Process(id='UseOfUreaFertiliser', produces=['AtmosphericCO2'], consumes=['Urea'], has_stock=False),
 Process(id='ProducingAmmoniumNitrate', produces=['AmmoniumNitrate'], consumes=['Ammonia', 'NitricAcid'], has_stock=False)]

In [13]:
model.objects

[Object(id='CO2', has_market=False),
 Object(id='Ammonia', has_market=True),
 Object(id='AmmoniumNitrate', has_market=True),
 Object(id='AtmosphericCO2', has_market=False),
 Object(id='CO2', has_market=True),
 Object(id='Coal', has_market=False),
 Object(id='Hydrogen', has_market=True),
 Object(id='NaturalGas', has_market=False),
 Object(id='NitricAcid', has_market=True),
 Object(id='Nitrogen', has_market=False),
 Object(id='OtherIndustrialGases', has_market=False),
 Object(id='PureOxygen', has_market=False),
 Object(id='Urea', has_market=True),
 Object(id='Water', has_market=False)]

In [16]:
flows = solution_to_flows(model, {demand_symbols["Urea"]: 5, demand_symbols["AmmoniumNitrate"]: 5, h2_coal_fraction: 0.5})
flows

,source,target,material,value
0,AmmoniaSynthesis,Ammonia,Ammonia,5.02500000000000
1,ExtractCO2FromAtmosphere,CO2,CO2,1.23750000000000
2,HydrogenSynthesisFromCoal,CO2,CO2,1.25625000000000
3,HydrogenSynthesisFromCoal,Hydrogen,Hydrogen,1.25625000000000
4,HydrogenSynthesisFromNaturalGas,CO2,CO2,1.25625000000000
5,HydrogenSynthesisFromNaturalGas,Hydrogen,Hydrogen,1.25625000000000
6,ProducingNitricAcid,NitricAcid,NitricAcid,3.93750000000000
7,ProducingNitricAcid,Water,Water,1.68750000000000
8,ProducingUrea,OtherIndustrialGases,OtherIndustrialGases,0.150000000000000
9,ProducingUrea,Urea,Urea,5


In [17]:
from ipysankeywidget import SankeyWidget
from ipywidgets import Layout
w = SankeyWidget(links=flows.to_dict(orient='records'), layout=Layout(width="1200", height="300"))
w.order = [
    ["NaturalGas", "Coal", "AtmosphericCO2"],
    ["HydrogenSynthesisFromNaturalGas", "HydrogenSynthesisFromCoal", "ExtractCO2FromAtmosphere"],
    ["Hydrogen", "CO2", "Nitrogen", "PureOxygen"],
    ["AmmoniaSynthesis", "ProducingNitricAcid"],
    ["Ammonia", "NitricAcid"],
    ["ProducingUrea", "ProducingAmmoniumNitrate"],
    ["Urea", "AmmoniumNitrate", "Water", "OtherIndustrialGases", "WasteOtherChemicals",],
    ["UseOfUreaFertiliser"],
]
w

SankeyWidget(layout=Layout(height='300', width='1200'), links=[{'source': 'AmmoniaSynthesis', 'target': 'Ammon…

In [18]:
from ipywidgets import interact

@interact(d1=(0., 10.), d2=(0., 10.), a3=(0., 1.))
def calc_flows(d1=5.0, d2=5.0, a3=0.5):
    flows = solution_to_flows(model, {demand_symbols["Urea"]: d1, demand_symbols["AmmoniumNitrate"]: d2, h2_coal_fraction: a3})
    w.links = flows.to_dict(orient='records')

interactive(children=(FloatSlider(value=5.0, description='d1', max=10.0), FloatSlider(value=5.0, description='…

In [8]:
from IPython.display import display

In [9]:
for j, value in model.Y.items():
    print("Process output: %s" % model.processes[j].id)
    display(value.value)
    print("History:", "\n".join(value.history))
    print("\n")

Process output: AmmoniaSynthesis


U_17*Z_2/(S_10*S_27) + U_14*U_87*Z_2/(S_10*S_27*S_84) + U_15*Z_1/(S_10*S_125)

History: set output of Ammonia from AmmoniaSynthesis = U_15*Z_1/S_125
set output of Ammonia from AmmoniaSynthesis = U_17*Z_2/S_27
set output of Ammonia from AmmoniaSynthesis = U_14*U_87*Z_2/(S_27*S_84)


Process output: ExtractCO2FromAtmosphere


Max(0, -S_42*(U_17*U_60*Z_2*a_1/(S_10*S_27*S_62) + U_14*U_60*U_87*Z_2*a_1/(S_10*S_27*S_62*S_84) + U_15*U_60*Z_1*a_1/(S_10*S_125*S_62)) - S_43*(U_17*U_60*Z_2*(1 - a_1)/(S_10*S_27*S_63) + U_14*U_60*U_87*Z_2*(1 - a_1)/(S_10*S_27*S_63*S_84) + U_15*U_60*Z_1*(1 - a_1)/(S_10*S_125*S_63)) + U_45*Z_1/S_125)/S_41

History: additional production to balance CO2


Process output: HydrogenSynthesisFromCoal


...

History: set output of Urea from ProducingUrea = Z_1


Process output: UseOfUreaFertiliser


Max(0, Z_1)/U_126

History: additional consumption to balance Urea


Process output: ProducingAmmoniumNitrate


Z_2/S_27

History: set output of AmmoniumNitrate from ProducingAmmoniumNitrate = Z_2


